# Description

The goal of this notebook is very simple: it just lists the content (traits/diseases) that belong to each cluster across all selected "best partitions". Although one would take a look at them here to check whether clusters of traits make sense, that analysis is carried out first by looking at the clustering trees (which are generated later). Then, this notebooks serves as a simple list with the content of the clusters.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

from utils import generate_result_set_name
import conf

# Settings

In [3]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

## Load data

In [4]:
INPUT_SUBSET = "umap"

In [5]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [6]:
DR_OPTIONS = {
    "n_components": 5,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [7]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0'

In [8]:
data_umap = pd.read_pickle(input_filepath)

In [9]:
data_umap.shape

(3749, 5)

In [10]:
data_umap.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
100001_raw-Food_weight,-8.437452,1.664409,9.233617,6.944210,16.781176
100002_raw-Energy,-8.388347,1.670845,9.251021,6.975690,16.698511
100003_raw-Protein,-8.424020,1.696144,9.226449,6.970763,16.768959
100004_raw-Fat,-8.393674,1.685311,9.258855,6.992673,16.728815
100005_raw-Carbohydrate,-8.427763,1.685172,9.234971,6.967204,16.751522


# Load best partitions

In [11]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [12]:
best_partitions = pd.read_pickle(input_file)

In [13]:
best_partitions.shape

(59, 4)

In [14]:
best_partitions.head()

,method,partition,ami_mean,selected
k,,,,
11,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301837,True
10,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301736,True
9,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.301299,True
8,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.300169,True
7,scc_020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.299448,True


# Analysis of clusterings

In [15]:
from IPython.display import HTML

In [16]:
def show_cluster_stats(clustering_data, selected_partition, selected_cluster):
    traits = [t for t in clustering_data[selected_partition == selected_cluster].index]
    display(len(traits))
    display(traits)

In [17]:
selected_k_values = best_partitions[best_partitions["selected"]].index.tolist()
selected_k_values.sort()
display(selected_k_values)

[4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 31,
 35,
 38,
 44,
 46,
 48]

In [18]:
for k in selected_k_values:
    display(HTML(f"<h2>Partition with k={k}</h2>"))
    display(best_partitions.loc[k])

    part = best_partitions.loc[k, "partition"]
    display(part.shape)

    part_stats = pd.Series(part).value_counts()
    display(part_stats)

    for cluster_number in part_stats.index.sort_values():
        display(HTML(f"<h3>Cluster {k}.{cluster_number}</h3>"))

        cluster_traits = data_umap[part == cluster_number].index
        display(len(cluster_traits))
        display(cluster_traits)

method                              eac_average_coassoc_matrix
partition    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
ami_mean                                               0.29643
selected                                                  True
Name: 4, dtype: object

(3749,)

2    3641
1      47
3      41
0      20
dtype: int64

20

Index(['3144_raw-Heel_Broadband_ultrasound_attenuation_direct_entry',
       '3147_raw-Heel_quantitative_ultrasound_index_QUI_direct_entry',
       '3148_raw-Heel_bone_mineral_density_BMD',
       '4101_raw-Heel_broadband_ultrasound_attenuation_left',
       '4104_raw-Heel_quantitative_ultrasound_index_QUI_direct_entry_left',
       '4105_raw-Heel_bone_mineral_density_BMD_left',
       '4106_raw-Heel_bone_mineral_density_BMD_Tscore_automated_left',
       '4120_raw-Heel_broadband_ultrasound_attenuation_right',
       '4123_raw-Heel_quantitative_ultrasound_index_QUI_direct_entry_right',
       '4124_raw-Heel_bone_mineral_density_BMD_right',
       '4125_raw-Heel_bone_mineral_density_BMD_Tscore_automated_right',
       '5096_raw-3mm_weak_meridian_left', '5097_raw-6mm_weak_meridian_left',
       '5098_raw-6mm_weak_meridian_right', '5099_raw-3mm_weak_meridian_right',
       '5132_raw-3mm_strong_meridian_right',
       '5133_raw-6mm_strong_meridian_right',
       '5134_raw-6mm_strong_meridi

...

Index(['100001_raw-Food_weight', '100002_raw-Energy', '100003_raw-Protein',
       '100004_raw-Fat', '100005_raw-Carbohydrate', '100006_raw-Saturated_fat',
       '100007_raw-Polyunsaturated_fat', '100008_raw-Total_sugars',
       '100009_raw-Englyst_dietary_fibre', '100011_raw-Iron',
       '100012_raw-Vitamin_B6', '100014_raw-Folate', '100015_raw-Vitamin_C',
       '100016_raw-Potassium', '100017_raw-Magnesium', '100019_raw-Carotene',
       '100023_raw-Starch', '100024_raw-Calcium', '100025_raw-Vitamin_E'],
      dtype='object')

1

Index(['lymphocyte count'], dtype='object')

118

Index(['20074_raw-Home_location_at_assessment_east_coordinate_rounded',
       '20075_raw-Home_location_at_assessment_north_coordinate_rounded',
       '20118_11-Home_area_population_density_urban_or_rural_Scotland_Large_Urban_Area',
       '22704_raw-Home_location_north_coordinate_rounded',
       '30220_raw-Basophill_percentage', 'abdominal injury',
       'abnormal delivery', 'alcohol dependence', 'appendicitis', 'arthritis',
       ...
       'tongue neoplasm', 'transient ischemic attack', 'tuberculosis',
       'type ii diabetes mellitus', 'ulcerative colitis',
       'urogenital neoplasm', 'urticaria', 'vascular disease', 'vasculitis',
       'wrist fracture'],
      dtype='object', length=118)